In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# AdaBoost

In [13]:
data = pd.read_excel("df.xlsx")
binary = {'M':1,'B':0}
data['diagnosis'] = data['diagnosis'].map(binary)

In [14]:
y = data['diagnosis']
X = data.drop(["diagnosis", "id"], axis = 1)

X_scaled = (X - np.min(X)) / (np.max(X) - np.min(X))
X = X_scaled

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.2, stratify = y)

In [16]:
model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state = 10, max_depth = 3, max_leaf_nodes = 4), n_estimators = 500, learning_rate = 1)

model_ada.fit(X_train, y_train)

y_pred_ab = model_ada.predict(X_test)

print(recall_score(y_test, y_pred_ab))
print('Adaptive Boosting Tuned Confusion Matrix:\n',confusion_matrix(y_test, y_pred_ab))

0.9285714285714286
Adaptive Boosting Tuned Confusion Matrix:
 [[70  2]
 [ 3 39]]


In [23]:
# Decision Tree
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, cross_val_predict
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
from sklearn import tree
import pydotplus

import time as time

In [17]:
data = pd.read_excel("df.xlsx")
data = data.drop('id', axis =1)
data.diagnosis = data.diagnosis.eq('M').mul(1)
data
y = data.diagnosis
X = data.drop("diagnosis", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

In [18]:
decisiontreemodel = DecisionTreeClassifier(random_state=1, max_depth = 6, max_leaf_nodes=3, max_features=2).fit(X_train, y_train)


In [25]:
# Random Forest
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import RandomForestClassifier


#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
import pydotplus
import time as tm

In [31]:
data = pd.read_excel('df.xlsx')
data_dummy = pd.get_dummies(data)
X = data_dummy.drop(['id', 'diagnosis_B','diagnosis_M'], axis = 1)
y = data_dummy['diagnosis_M']

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.3, random_state=45)

In [32]:
param_grid = {
    'n_estimators': range(20,25,1),
    'max_depth': range(5,8,1),
    'max_leaf_nodes': range(10,14,1),
    'max_features': range(8,12,1)
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=45), param_grid, scoring=['accuracy','recall'], 
                           refit="recall", cv=5, n_jobs=-1, verbose = True)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=45), n_jobs=-1,
             param_grid={'max_depth': range(5, 8), 'max_features': range(8, 12),
                         'max_leaf_nodes': range(10, 14),
                         'n_estimators': range(20, 25)},
             refit='recall', scoring=['accuracy', 'recall'], verbose=True)

In [33]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score

In [35]:
model_rf = grid_search.best_estimator_
feature_importances = np.mean([
    tree.feature_importances_ for tree in best_model.estimators_
], axis=0)

model_rf

RandomForestClassifier(max_depth=6, max_features=10, max_leaf_nodes=11,
                       n_estimators=21, random_state=45)

In [ ]:
#Ensemble

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth

In [38]:
data = pd.read_excel("df.xlsx")
data = data.drop('id', axis =1)
data.diagnosis = data.diagnosis.eq('M').mul(1)
data
y = data.diagnosis
X = data.drop("diagnosis", axis = 1)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 45)

In [ ]:
from sklearn.metrics import make_scorer, recall_score

In [44]:
scorer = make_scorer(recall_score)
ensemble_model = StackingClassifier(estimators=[('ada',model_ada),('rf',model_rf),('dt',decisiontreemodel)],
                                   final_estimator=RandomForestClassifier(n_estimators=500, max_features=1,
                                    random_state=1,oob_score=True),n_jobs=-1,
                                   cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1))
ensemble_model.fit(Xtrain,ytrain)
ensemble_model.score(Xtest, ytest)

0.9824561403508771